In [1]:
import pandas as pd
import scipy.spatial.distance as sci_dist
import matplotlib
matplotlib.use('QtAgg')
import seaborn as sns
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
import os
np.random.seed(0)
matplotlib.rcParams['pdf.fonttype'] = 42

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

data_folder = '/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_2/exports'
region_data = '/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/processedData/Region_Data_with_Minimal_Values.csv'
cell_by_gene_path = data_folder + '/cell_by_gene_matrix_dilate10_20240718_withbarcodes_atleast3.csv'
transcripts_path = data_folder + '/decode_1_withcoordinates.csv'
coords_path = data_folder + '/segmentation_1_nuclei_cellattributes.csv'
output_folder = data_folder + '/processedData'
cell_by_gene = pd.read_csv(cell_by_gene_path)


In [2]:
pixel2um = 107.11 / 1000000  # Convert pixel to micrometers
combined_df = pd.DataFrame()
region_data = pd.read_csv(region_data)
for j in range(1, 4):  # Adjust range as needed
    # Dynamically construct the path
    data_folder = f"/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_{j}/exports"
    cell_by_gene_path = os.path.join(data_folder, 'cell_by_gene_matrix_dilate10_20240718_withbarcodes_clustering_10bcs_0.4thresh.csv')
    print(data_folder)
    # Load the cell by gene data
    cell_by_gene = pd.read_csv(cell_by_gene_path, index_col=0)
    
    # Get reference X and Y from region_data as scalar values
    ref_x = region_data.loc[region_data['Region'] == f"Region {j}", "Min X (mm)"].iloc[0]
    ref_y = region_data.loc[region_data['Region'] == f"Region {j}", "Min Y (mm)"].iloc[0]
    
    # Correct the coordinates
    cell_by_gene['center_x'] = (cell_by_gene['center_x'] * pixel2um + ref_x)*1000
    cell_by_gene['center_y'] = (cell_by_gene['center_y'] * pixel2um + ref_y)*1000
    
    # Adjust the cell_id to make them unique across regions
    cell_by_gene['cell_id'] = cell_by_gene['cell_id'] + j * 100000
    
    # Concatenate to the combined DataFrame
    combined_df = pd.concat([combined_df, cell_by_gene], ignore_index=True)


/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_1/exports
/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_2/exports
/Users/yaelheyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/roi_3/exports


In [3]:
barcode_indices = {}
cell_by_gene = combined_df
# Loop through each barcode and add its index to the appropriate list
for index, barcode in enumerate(cell_by_gene['barcode_names']):
    # Convert NaN values to a string representation to handle them as unique keys
    barcode_key = barcode if barcode == barcode else 'NaN'
    if barcode_key in barcode_indices:
        barcode_indices[barcode_key].append(index)
    else:
        barcode_indices[barcode_key] = [index]

# Convert the dictionary to a list of lists (containing the indices)
indices_list = list(barcode_indices.values())
for ids in indices_list:
    print('ids')
    print(ids)
    for id in ids:
        if(id>0):
            print(cell_by_gene.loc[id,"barcode_names"])
unique_index_list = [
    cell_by_gene.loc[ids[0], ["barcode_names", "n_called_barcodes"]]
    for ids in indices_list
    if pd.notna(cell_by_gene.loc[ids[0], "barcode_names"])
]
unique_index_df = pd.DataFrame(unique_index_list, columns=["barcode_names", "n_called_barcodes"])
# Now you can access the column:
unique_index_df["barcode_names"]
print('Percentage zero barcodes: ' + str(np.sum(pd.isna(cell_by_gene['barcode_names'])) / len(cell_by_gene)))


ids
[0, 1, 3, 4, 5, 6, 9, 10, 13, 14, 15, 17, 18, 20, 22, 23, 26, 27, 31, 32, 34, 35, 36, 37, 40, 42, 43, 44, 45, 48, 51, 57, 59, 60, 61, 63, 65, 71, 72, 74, 75, 79, 80, 82, 92, 98, 101, 107, 108, 109, 110, 113, 114, 117, 118, 122, 129, 130, 133, 135, 137, 141, 152, 153, 162, 168, 169, 170, 171, 172, 173, 174, 176, 177, 189, 190, 191, 194, 195, 196, 205, 209, 213, 214, 216, 217, 218, 220, 222, 223, 224, 226, 229, 231, 234, 236, 239, 243, 246, 247, 252, 253, 257, 258, 259, 261, 263, 264, 266, 267, 268, 271, 273, 279, 283, 285, 288, 290, 292, 296, 297, 300, 301, 304, 309, 317, 323, 326, 334, 335, 340, 341, 347, 348, 350, 351, 352, 354, 357, 360, 363, 365, 366, 367, 370, 380, 381, 382, 384, 386, 389, 391, 393, 394, 397, 398, 399, 403, 404, 411, 412, 413, 414, 415, 417, 424, 429, 434, 435, 441, 442, 443, 444, 445, 446, 447, 451, 462, 463, 466, 471, 476, 480, 481, 482, 485, 490, 491, 493, 495, 496, 498, 500, 503, 507, 510, 514, 515, 517, 528, 533, 534, 541, 542, 543, 544, 551, 552, 554, 555

In [4]:
1-np.sum(pd.isna(cell_by_gene['barcode_names'])) / len(cell_by_gene)

np.float64(0.8910170007391626)

In [5]:
# data =unique_index_df["n_called_barcodes"]  #if using the grouping logic
data = cell_by_gene["n_called_barcodes"] #if going for direct count
# Estimate the lambda (mean) of the Poisson distribution
lambda_estimate = np.mean(data)
lambda_estimate 
# Calculate the empirical PMF
unique_values, counts = np.unique(data, return_counts=True)
empirical_pmf = counts / len(data)

# Theoretical Poisson PMF
x_values = np.arange(0, max(data) + 1)  # Range of possible values
theoretical_pmf = poisson.pmf(x_values, mu=lambda_estimate)

# Plotting the PMF
plt.figure(figsize=(2,2))

# Empirical PMF as a bar plot
plt.bar(unique_values, empirical_pmf, width=0.4, label='Empirical PMF', alpha=0.7, align='center')

# Theoretical Poisson PMF as a line plot
# plt.plot(x_values, theoretical_pmf, 'go-', label=f'Theoretical Poisson PMF (λ={lambda_estimate:.1f})')
plt.xlim([-1,16])
# Labels and legend
plt.xlabel('Number of barcodes per cell')
plt.ylabel('Probability')
plt.title('Empirical vs theoretical PDF')
plt.legend(bbox_to_anchor=(1.05, 0.5),
          loc='center left',
          borderaxespad=0)
plt.grid(True)
plt.savefig(output_folder + '/empirical vs predicted pdf in vitro unclustered.pdf',bbox_inches='tight')
plt.show()


In [ ]:
cdf = np.empty(0)
for i in range(len(unique_values)):
    cdf = np.append(cdf, np.sum(empirical_pmf[i:]))
cdf_df = pd.DataFrame({ 'num barcodes': unique_values,'cdf': cdf}) 
# cdf_df.to_csv(os.path.join(output_folder, 'cdf vs unique number of barcodes.csv'))